In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import camb
from camb.sources import GaussianSourceWindow, SplinedSourceWindow
import camb.correlations
from camb import model, initialpower
import numpy as np
import astropy.table
import pandas as pd
import math
import pickle
from astroML.datasets import fetch_sdss_specgals
from scipy.optimize import fmin

import SDSS_treecorr as stc

In [2]:
data = fetch_sdss_specgals()
m_max = 17.7
# redshift and magnitude cuts
data = data[data['z'] > 0.08]
data = data[data['z'] < 0.12]
data = data[data['petroMag_r'] < m_max]
# RA/DEC cuts
RAmin, RAmax = 140, 220 
DECmin, DECmax = 5, 45
data = data[data['ra'] < RAmax] 
data = data[data['ra'] > RAmin] 
data = data[data['dec'] < DECmax] 
data = data[data['dec'] > DECmin]
ur = data['modelMag_u'] - data['modelMag_r'] 
flag_red = (ur > 2.22)
flag_blue = ~flag_red
data_red = data[flag_red] 
data_blue = data[flag_blue]
print("data size:")
print(" red gals: ", len(data_red)) 
print(" blue gals:", len(data_blue))

data size:
 red gals:  38017
 blue gals: 16883


In [3]:
Nz, be = np.histogram(data['z'][flag_red], bins=8, range=(0.05,0.15))

In [4]:
def getcorrTree(data):
    corr, bincenters, cov = stc.calccorr(data)
    return [corr, math.e**bincenters, cov]

In [5]:
def getcorrCAMB(centers, Om, bias):
    lmax = 7500
    pars = camb.CAMBparams() # Set up the CAMB parameters
    h=0.675 # Planck value for h (Hubble parameter)
    Ob = 0.044 # Planck value for Omega_b (Baryon energy density)
    #Om = 0.31 # Planck value for Omega_m (Matter energy density)
    Oc = Om-Ob # Value for Omega_c (Cold dark matter energy density)
    As=2e-9 # Amplitude of initial fluctuations
    ns=0.965 # Scalar index
    pars.set_cosmology(H0=100*h, ombh2=Ob*h**2, omch2=Oc*h**2) # This sets the cosmological parameters
    pars.InitPower.set_params(As=As, ns=ns) # This also sets the cosmological parameters
    pars.set_for_lmax(lmax, lens_potential_accuracy=1) # Set the maximum ell
    #set Want_CMB to true if you also want CMB spectra or correlations
    pars.Want_CMB = False # We don't want the CMB
    #NonLinear_both or NonLinear_lens will use non-linear corrections
    pars.NonLinear = model.NonLinear_both # We want non-linear corrections
    #Set up W(z) window functions, later labelled W1, W2.
    zs = 0.5*(be[1:] + be[:-1]) #z # Range of zs
    W = Nz # Window function
    pars.SourceWindows = [SplinedSourceWindow(source_type='counts', bias=bias, z=zs, W=W)] # Set up the window function
    
    results = camb.get_results(pars)
    cls = results.get_source_cls_dict()
    ls=  np.arange(2, lmax+1)
    
    angles = centers #np.logspace(-2, 1) # Angles from 0.01 to 10 deg
    x = np.cos(np.radians(angles)) # Convert them to radians and compute cosine to passs to CAMB
    cls_in = np.array([cls['W1xW1'][1:lmax+1], np.zeros(lmax), np.zeros(lmax), np.zeros(lmax)]).T
    #cl2corr needs TT (temperature/density), EE (E-mode), BB (B-mode), TE (Temperature-polarization cross correlation) -> we only care about TT
    w_camb = camb.correlations.cl2corr(cls_in, x);
    
    return w_camb

In [6]:
def biasfunc2(b, corr_tree, corr_camb, inv_cov):
    return np.matmul((corr_tree- b**2*corr_camb), np.matmul(inv_cov,(corr_tree- b**2*corr_camb)))

In [16]:
def findb(data):
    corr, centers, cov = getcorrTree(data)
    w_camb = getcorrCAMB(centers, 0.31,1)
    
    corr_tree = corr[0]
    corr_camb = w_camb[:,0]
    
    inv_cov = np.linalg.inv(cov)
    
    result = fmin(biasfunc2, 0, args = (corr_tree, corr_camb, inv_cov), disp=False);
    
    return result[0]
    

In [8]:
corr1 = getcorrCAMB(centers, 0.3, 1)[:,0]

NameError: name 'centers' is not defined

In [10]:
corr, centers, cov = getcorrTree(data)
#w_camb = getcorrCAMB(centers)
w_camb = getcorrCAMB(centers, 0.3, 1)

In [11]:
corr_tree = corr[0]
corr_camb = w_camb[:,0]

In [57]:
"""

First let's look at the simplified case:

minimum = (corr_tree - (b**2)*(corr_camb))**2

... where we're trying to find a b that produces this minimum.

"""

def biasfunc1(b):
    return np.matmul(corr_tree - (b**2)*(corr_camb), corr_tree - (b**2)*(corr_camb))

In [58]:
biasfunc1(1)

4.7659774079960515

In [59]:
fmin(biasfunc1, 0)

Optimization terminated successfully.
         Current function value: 0.566635
         Iterations: 25
         Function evaluations: 50


array([1.2299375])

In [60]:
biasfunc1(1.2464375)

0.5934078970206809

In [61]:
"""
Alright, now with 

(data- b**2*theory)* Cov^{-1}*(data- b**2*theory) = minimum

Cov() is  the covariance matrix.

"""

inv_cov = np.linalg.inv(cov)

In [70]:
def biasfunc2(b, corr_tree, corr_camb):
    return np.matmul((corr_tree- b**2*corr_camb), np.matmul(inv_cov,(corr_tree- b**2*corr_camb)))

In [12]:
biasfunc2(1)

TypeError: biasfunc2() missing 3 required positional arguments: 'corr_tree', 'corr_camb', and 'inv_cov'

In [14]:
#result = fmin(biasfunc2, 0, args = (corr_tree, corr_camb))

In [69]:
biasfunc2(1.194625)

132.85749477688552

In [77]:
result[0]

1.2081250000000012

In [91]:
"""
Now for red and blue:

"""

print(findb(data_red))
print(findb(data_blue))
print(findb(data))


0.7083125000000008
0.3662500000000003
0.5868750000000005


In [18]:
print(findb(data))

1.2024375000000012


In [19]:
print(findb(data_red))
print(findb(data_blue))

1.4464375000000014
0.8197500000000009
